In [45]:
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
import torch

In [46]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [47]:
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [48]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [49]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [50]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [51]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [52]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

I am not here
{'spec': tensor([[[4.4112e+02, 1.1000e+00],
         [9.1055e+01, 9.1700e-01],
         [1.0507e+02, 8.6500e-01],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.4911e+02, 1.1000e+00],
         [5.2018e+01, 6.4900e-03],
         [5.3039e+01, 1.9750e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.5212e+02, 1.1000e+00],
         [3.2110e+02, 1.0000e-02],
         [3.2815e+02, 7.0000e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        ...,

        [[3.0904e+02, 1.1000e+00],
         [2.8706e+02, 2.0000e-02],
         [3.0919e+02, 3.7200e-01],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[2.9109e+02, 1.1000e+00],
         [9.5048e+01, 1.

In [53]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256,  23, 256, 101, 256, 256, 256, 256,  67, 256, 256, 256])


In [54]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [55]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [56]:
tmp_original['precursor_mz']

tensor([441.1200, 349.1132, 352.1200, 323.1177, 535.3272, 321.0192, 138.0550,
        411.9655, 524.3720, 309.0400, 291.0863, 351.1440])

In [57]:
tmp_original['adduct'], 

(['[M+Na]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+'],)

In [58]:
tmp_original['mol_freq']

tensor([121.,   3.,  23.,   3.,  10.,  42.,  48.,   3., 200., 141., 427.,   8.])

In [59]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [60]:
sum(tmp_original['labels'])

tensor(12)

In [61]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  279,  535,  636,  892, 1148, 1404, 1660, 1727, 1983, 2239])

In [62]:
len(tmp_original['labels'])

2495

In [63]:
tmp_original['batch_ptr']

tensor([256,  23, 256, 101, 256, 256, 256, 256,  67, 256, 256, 256])

In [64]:
sum(tmp_original['batch_ptr'])

tensor(2495)

In [65]:
len(tmp_original['candidates'])

2495

In [66]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [67]:
len(tmp_original['candidates_smiles'])

2495

# MSnRetrieval

In [20]:
from massspecgym.featurize import SpectrumFeaturizer

In [21]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [22]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [23]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 15674
Dataset length: 15674


In [24]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [25]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 11883
Val dataset size: 1899


In [26]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

in collate_fn
{'spec': DataBatch(x=[133, 1039], edge_index=[2, 121], batch=[133], ptr=[13]), 'mol': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'precursor_mz': tensor([297.1121, 331.1451, 369.1782, 386.2326, 455.0955, 470.0839, 517.1340,
        351.1358, 327.1551, 624.2673, 377.2686, 308.1030]), 'adduct': ['[M+H]+', '[M+Na]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H-2H2O]+', '[M+H]+'], 'identifier': ['0073967_0000000', '0029647_0000000', '0073834_0000000', '0014621_0000000', '0025270_0000000', '0036154_0000000', '0005041_0000000', '0005023_0000000', '0004557_0000000', '0082596_0000000', '0002898_0000000', '0091031_0000000'], 'mol_freq': tensor([1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 3., 1.]), 'smiles': ['C1COC2=C(O1)C=CC(=C2)C(=O)C

In [27]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256, 244, 256, 256, 150, 256,  27, 256, 256])


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [28]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [29]:
tmp_msn['spec'] 

DataBatch(x=[133, 1039], edge_index=[2, 121], batch=[133], ptr=[13])

In [30]:
tmp_msn['precursor_mz']

tensor([297.1121, 331.1451, 369.1782, 386.2326, 455.0955, 470.0839, 517.1340,
        351.1358, 327.1551, 624.2673, 377.2686, 308.1030])

In [31]:
tmp_msn['mol_freq']

tensor([1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 3., 1.])

In [32]:
tmp_msn['identifier']

['0073967_0000000',
 '0029647_0000000',
 '0073834_0000000',
 '0014621_0000000',
 '0025270_0000000',
 '0036154_0000000',
 '0005041_0000000',
 '0005023_0000000',
 '0004557_0000000',
 '0082596_0000000',
 '0002898_0000000',
 '0091031_0000000']

In [33]:
tmp_msn['adduct']

['[M+H]+',
 '[M+Na]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H-2H2O]+',
 '[M+H]+']

In [34]:
tmp_msn['mol'].shape

torch.Size([12, 4096])

In [35]:
tmp_msn['smiles']

['C1COC2=C(O1)C=CC(=C2)C(=O)CCC(=O)C3=CC=CC=C3',
 'C1CCC(C1)(CNC(=O)NCC2CCCO2)C3=CSC=C3',
 'CC1=CC(=NO1)C2=NN=CC3=C(C(=NN32)OCC4=NC=NN4C)C(C)(C)C',
 'C[C@@H]1CCC2[C@@]([C@@]13CC4=C(C=C5C(=C4O3)CNC5=O)O)(CC[C@H](C2(C)C)O)C',
 'CC1=NN=C2N1N=C(S2)C3=CC(=CC=C3)NC(=S)NC(=O)C4=C(C=CC=C4OC)OC',
 'CC1=CC=CC=C1C2=NOC(=N2)C3=C(C=CS3)S(=O)(=O)N(C)C4=CC5=C(C=C4)OCCO5',
 'COC1=CC=C(C=C1)C2=COC3=C(C2=O)C=CC(=C3)O[C@H]4[C@@H]([C@H]([C@@H]([C@H](O4)COC(=O)CC(=O)O)O)O)O',
 'CCC(C)/C=C(\\C)/C=C/C1=CC2=C(C(=O)C(C(C2=CO1)O)(C)O)Cl',
 'CC(C(C12CC1CC=CC2=O)O)NC(=O)C(CC(C(=O)O)O)N',
 'CC1=CC(=C(C=C1SC2=CN=C(S2)NC(=O)C3=CC=C(C=C3)CNC(C)C(C)(C)C)C(=O)N4CCN(CC4)C(=O)C)OC',
 'CC(CC(C)C(C)O)CC(CC(C)/C=C(\\C)/C=C(/CC(C)C(=O)O)\\C(=O)O)CO',
 'COC1=CC2=C(C=C1)NC(=C2C3=C(C4=CC=CC=C4N3)N=O)O']

In [36]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [37]:
sum(tmp_msn['labels'])

tensor(12)

In [38]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1268, 1524, 1780, 1930, 2186, 2213, 2469])

In [39]:
len(tmp_msn['labels'])

2725

In [40]:
tmp_msn['batch_ptr']

tensor([256, 256, 256, 256, 244, 256, 256, 150, 256,  27, 256, 256])

In [41]:
sum(tmp_msn['batch_ptr'])

tensor(2725)

In [42]:
len(tmp_msn['candidates'])

2725

In [43]:
tmp_msn['candidates'][5].shape

torch.Size([4096])

### Compare Shapes

In [68]:
tmp_msn.keys() == tmp_original.keys()

True

In [69]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [70]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

True

In [71]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(2725), 2725)

In [72]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(2495), 2495)

In [73]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [74]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [75]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [76]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [77]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [78]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [79]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [80]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [81]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [82]:
len(tmp_msn['candidates_smiles'])

2725